In [ ]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import undetected_chromedriver as uc
import locale

import calendar
import random

In [ ]:

json_pages_info = {"pages": []}

options = Options()
options.add_argument("--headless") 
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

articles_count = 0

pages = 5

url = f"https://www.latercera.com/search/?q=Estallido%20Social%20Chile"

driver.get(url)

time.sleep(10)

for page in range(2, pages + 1):

	elements = driver.find_element(By.CLASS_NAME, "gsc-expansionArea")
	articles = elements.find_elements(By.CLASS_NAME, "gsc-webResult")
	print(f"{len(articles)} articles found")

	for article in articles:
		print("Analyzing article...")

		date = article.find_element(By.CLASS_NAME, "gsc-table-result").find_element(By.CLASS_NAME, "gsc-table-cell-snippet-close").find_element(By.CLASS_NAME, "gs-bidi-start-align")
		print(date.text)
		originalDate = date.text[:10] if date else "not found"
		splitDate = originalDate.split("-")

		if(len(splitDate) != 3 or date == None):
			print("Fecha no encontrada")
			continue
		
		timestamp = calendar.timegm(time.strptime(f'{splitDate[2]}-{splitDate[1]}-{splitDate[0]} 00:00:00', '%Y-%m-%d %H:%M:%S'))

		if (timestamp < 1573786800 or timestamp > 1702782000):
			print("La noticia no corresponde a la fecha solicitada")
			continue

		print(splitDate)

		try:

			link = article.find_element(By.CLASS_NAME, "gs-title").find_element(By.CLASS_NAME, "gs-title").get_attribute("href")
			title = article.find_element(By.CLASS_NAME, "gs-title").find_element(By.CLASS_NAME, "gs-title").text
			image_link = article.find_element(By.CLASS_NAME, "gs-image").find_element(By.CLASS_NAME, "gs-image").get_attribute("src")
			description = article.find_element(By.CLASS_NAME, "gsc-table-result").find_element(By.CLASS_NAME, "gsc-table-cell-snippet-close").find_element(By.CLASS_NAME, "gs-bidi-start-align")
			description = description.text[10:] if description else "not found"
		except Exception as e:
			print("Error al obtener los datos del artículo: ", e)
			continue

		link_info = {
			"newscast" : "La Tercera",
			"title": title,
			"description": description,
			"category": "The site does not provide a category",
			"date": originalDate,
			"image_link": image_link,
			"author": "not found initially",
			"link": link,
		}
		articles_count += 1
		print("Añadiendo articulo")
		json_pages_info["pages"].append(link_info)

	buttons = driver.find_element(By.CLASS_NAME, "gsc-cursor")
	buttons = buttons.find_elements(By.CLASS_NAME, "gsc-cursor-page")
	print(len(buttons))

	button_found = False

	for button in buttons:
		if(button.text == str(page)):
			print("Button found: " + button.text)
			button.click()
			button_found = True
			time.sleep(2)
			break

	if(not button_found):
			print("The button for page " + str(page) + " was not found.")
			break

driver.quit()

with open(f"../archive/temp/laTercera_pages.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


In [ ]:
import pyautogui
import random

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

# json_pages_info = {"pages": []}

pages = 32

data_extracted = True

re = False

for page in range(22, pages):

	first = True
	while (re or first):
		first = False
		
		try: 
			time.sleep(1)

			if (not re): 
				options = uc.ChromeOptions()
				options.add_argument("--no-sandbox")
				options.add_argument("--disable-blink-features=AutomationControlled")

				driver = uc.Chrome(options=options)

				url = f'https://www.google.com/search?q=%22Estallido+social%22+site%3Awww.latercera.com&tbs=cdr:1,cd_min:11/15/2019,cd_max:12/17/2023&start={page * 10}'
				driver.get(url)

			re = False
			time.sleep(random.uniform(6, 12))

			articles_section = driver.find_element(By.CLASS_NAME, "dURPMd")
			articles = articles_section.find_elements(By.CLASS_NAME, "MjjYud")

			location = articles_section.location
			size = articles_section.size
			x = location['x'] + random.randint(5, size['width'] - 5)
			y = location['y'] + random.randint(5, size['height'] - 5)
			duration = random.uniform(0.8, 2.0)
			pyautogui.moveTo(x, y, duration=duration)

			for article in articles:
				print("revisando artículo...")
				try: 
					originalDate = article.find_element(By.CLASS_NAME, "YrbPuc").find_element(By.TAG_NAME, "span").text
					for es, en in months.items():
						if es in originalDate:
								originalDate = originalDate.replace(es, en)
								break
						
					date_epoch = int(time.mktime(time.strptime(originalDate, "%d %b %Y")))

					if (date_epoch < 1573786800 or date_epoch > 1702782000):
						print("La noticia no corresponde a la fecha solicitada")
						continue

					try:
						title = article.find_element(By.TAG_NAME, "h3").text
						description = article.find_element(By.CLASS_NAME, "kb0PBd ").find_elements(By.TAG_NAME, "span")[1].text
						link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

						link_info = {
							"newscast" : "El Mostrador",
							"title": title,
							"description": description,
							"category": "The site does not provide a category",
							"date": originalDate,
							"image_link": "not found initially",
							"author": "not found initially",
							"link": link,
						}

						print("información incluida!")
						json_pages_info["pages"].append(link_info)

					except Exception as e:
						print(f"Error al extraer información del artículo: {e}")
						continue

				except Exception as e:
					print(f"Error al procesar el artículo: {e}")
					continue
		except Exception as e:
			print(f"Error al procesar el resultado de busqueda. Página: {page}")
			print(f"Sitio: {url}")
			print(e)
			response = input("Reintentar? y/n")
			if response.lower() == 'y':
				print("Reintentando.")
				re = True
			else: 
				print("Cancelando...")
				re = False
		
		if (not re):
			driver.quit()

print("Almacenando información en el archivo JSON...")
with open(f"../archive/temp/la_tercera_080725.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


# with open(f"output.html", "w", encoding="utf-8") as f:
# 	f.write(driver.page_source)

In [ ]:
import json

all_data = {"pages": []}
coincidence = 0
clean_data_links = []

clean_data = {
	"pages": []
}

with open("../archive/temp/la_tercera_090725.json", 'r', encoding='utf-8') as file:
	all_data = json.load(file)

all_links = [page["link"] for page in all_data["pages"]]

coincidence_boolean = False

print(f"Total links in raw data: {len(all_data['pages'])}")

for page in all_data["pages"]:
	for link in clean_data_links:
		if page["link"] in link:
			print(f"Coincidencia encontrada: {page['link']} con link {link}")
			coincidence += 1
			coincidence_boolean = True
			break
	if not coincidence_boolean:
		clean_data["pages"].append(page)
		clean_data_links.append(page["link"])
	coincidence_boolean = False

print(f"Total coincidences: {coincidence}")
print(f"Total unique links: {len(clean_data['pages'])}")

with open(f"../archive/temp/la_tercera_090725_cleanData.json", 'w', encoding='utf-8') as file:
	json.dump(clean_data, file, ensure_ascii=False, indent=4)


In [ ]:
import json

data = {}
clean_data = {"pages": []}

with open("../archive/temp/la_tercera_080725_raw.json", 'r', encoding='utf-8') as file:
	data = json.load(file)

for page in data["pages"]:
	page["newscast"] = "La Tercera"
	clean_data["pages"].append(page)

with open(f"../archive/temp/la_tercera_090725.json", 'w', encoding='utf-8') as file:
	json.dump(clean_data, file, ensure_ascii=False, indent=4)

In [ ]:
import random

extracted_pages = {"pages": []}
extracted_pages_with_content = {"pages": []}

with open("../archive/pages_extracted/la_tercera/la_tercera_090725_cleanData.json", 'r', encoding='utf-8') as file:
	extracted_pages = json.load(file)

options = Options()
options.add_argument("--headless") 
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-software-rasterizer")
options.add_argument("--enable-features=NetworkService,NetworkServiceInProcess")
options.add_argument("--browser-version=139")
driver = webdriver.Chrome(options=options)

data_extracted = True
re = False

for page_index, page in enumerate(extracted_pages["pages"]):
	print(f"Link - {page['link']}" )

	url = page['link']

	first = True

	while (re or first):
		first = False
		
		try: 
			time.sleep(1)

			if (not re): 
				options = uc.ChromeOptions()
				options.add_argument("--no-sandbox")
				options.add_argument("--disable-blink-features=AutomationControlled")

				driver = uc.Chrome(options=options)

				driver.get(url)

			re = False
			time.sleep(random.uniform(6, 12))

			try: 
				title = driver.find_element(By.TAG_NAME, "h1").text
				extracted_pages_with_content["pages"].append({"title": title, "page": page})
				print(title)
			except Exception as e:
				print(f"Error al procesar el artículo: {e}")
				continue

		except Exception as e:
			print(f"Error al procesar. Página: {page_index}")
			print(f"Sitio: {url}")
			print(e)
			response = input("Reintentar? y/n")
			if response.lower() == 'y':
				print("Reintentando.")
				re = True
			else: 
				print("Cancelando...")
				re = False
		
		if (not re):
			driver.quit()

	print("Almacenando información en el archivo JSON...")
	with open(f"../archive/temp/ciper_pages_with_content.json", 'w', encoding='utf-8') as file:
		json.dump(extracted_pages_with_content, file, ensure_ascii=False, indent=4)

	time.sleep(random.uniform(6, 12))

In [ ]:

extracted_pages = {"pages": []}
extracted_pages_with_content = {"pages": []}
last_extracted_pages_with_content = {"pages": []}

total_pages = 0
pages_succeeded = 0
pages_failed = 0
error_occurred = False
driver = None
limit_of_pages = "all"
start_page = 0

content = ""
author = ""
description = ""
raw_json_content = {}

auto_response_count = 0

test_specific_url = ""

def get_response():
	return input("Retry? y/n: ")

print("[INFO] Loading extracted pages from JSON file...")
with open("../archive/temp/pages_with_content/la_tercera/la_tercera_pages_to_check.json", 'r', encoding='utf-8') as file:
	extracted_pages = json.load(file)
	print(f"[INFO] {len(extracted_pages['pages'])} pages loaded.")
	if limit_of_pages == "all":
		limit_of_pages = len(extracted_pages["pages"])
		print(f"[INFO] Limit of pages set to {limit_of_pages}")

def recursive_search(element):
	element_tag_name = element.tag_name
	print(element_tag_name)
	print(element.text)
	if element_tag_name == "p" or element_tag_name == "span" or element_tag_name == "h2" or element_tag_name == "h3" or element_tag_name == "h4" or element_tag_name == "a":
		element.text
		return element.text.strip()
	
	children = element.find_elements(By.XPATH, ".//*")
	for child in children:
		result = recursive_search(child)
		if result:
			return result
	return ""


# with open("../archive/temp/pages_with_content/la_tercera_with_content_reference.json", 'r', encoding='utf-8') as file:
# 	last_extracted_pages_with_content = json.load(file)
# 	print(f"[INFO] {len(last_extracted_pages_with_content['pages'])} pages loaded.")


if start_page >= len(extracted_pages["pages"]):
	print(f"[ERROR] start_page is out of range. The maximum page index is {len(extracted_pages['pages']) - 1}.")
	raise SystemExit

def extract_from_script():
		print("[INFO] Trying to extract content from <script> tags...")
		data = {"content": "", "description": "", "raw_json_content": ""}

		scripts = driver.find_elements(By.XPATH, "//head/script")
		data_in_script = {}
		content_extracted_by_json = False
		for script in scripts: 
			script_content = script.get_attribute("innerHTML").strip()

			# Intentar interpretarlo como JSON
			is_json = False
			content_extracted_by_json = False
			try:
					data_in_script = json.loads(script_content)   # si es JSON válido, lo convierte en dict/list
					test = data_in_script["articleBody"]
					if test is not None:
						print("[INFO] Content found in <script> tag.")
					is_json = True
			except json.JSONDecodeError:
					is_json = False

			except Exception as e:
					is_json = False
			
			try:
				if is_json:
						data["content"] = data_in_script["articleBody"]
						data["raw_json_content"] = data_in_script
						
						try:
							data["description"] = data_in_script["description"]
						except Exception as e:
							print("[ERROR] A error occurred while extracting description", flush=True)
							data["description"] = "not found"


						content_extracted_by_json = True
						print("[INFO] The content has been extracted. Method: JSON", flush=True)
						break
			except Exception as e:
					print(f"[ERROR] A error occurred while extracting content from JSON script: {e}", flush=True)
					content_extracted_by_json = False

		return content_extracted_by_json, data

def extract_from_DOM():
	print("[INFO] Trying to extract content from DOM...")
	content = ""
	author = ""
	description = ""

	# paragraph = content_container.find_elements(By.TAG_NAME, "p") if content_container else None
	paragraph_elements = body.find_elements(By.CLASS_NAME, "article-body__paragraph")
	content = ""

	for element in paragraph_elements:
		p = recursive_search(element)
		if "Volver al Home" in p or "También puedes leer" in p:
			continue
		content += p + "\n"

	try:
		description = body.find_element(By.CLASS_NAME, "article-head__subtitle").text.strip()
	except:
		description = "not found"

	print("[INFO] The content has been extracted. Method: DOM", flush=True)
	if len(content) < 100:
			print("[WARNING] The extracted content is too short.", flush=True)
			print(f"[WARNING] Content: {content}", flush=True)
	return content, author, description

data_extracted = True
re = False
auto_re = False

def get_driver():
	options = uc.ChromeOptions()
	options.add_argument("--no-sandbox")
	options.add_argument("--disable-blink-features=AutomationControlled")

	driver = uc.Chrome(options=options)
	driver.set_page_load_timeout(15)
	return driver

print("[INFO] Starting extraction process...")
for page_index, page in enumerate(extracted_pages["pages"]):

		if test_specific_url == "":
			if total_pages >= limit_of_pages:
					print("[INFO] Limit of pages reached.")
					break

			if "content" not in page.keys():
					page["content"] = "The content has not been extracted yet"

			if page_index < start_page:
					# if page_index < len(last_extracted_pages_with_content["pages"]):
					# 		extracted_pages_with_content["pages"].append(last_extracted_pages_with_content["pages"][page_index])
					# else:
					# 	extracted_pages_with_content["pages"].append(page)
					print("[INFO] Skipping page: ", page_index)
					continue
			if page["content"] != "The content has not been extracted yet" and page["content"] != "": 
					print(f"[INFO] Page {page_index} already has content. Skipping...")
					extracted_pages_with_content["pages"].append(page)
					continue
		else:
			print("[INFO] Processing specific page: ", test_specific_url)
			for page_aux in extracted_pages["pages"]:
				if page_aux["link"] == test_specific_url:
					page = page_aux
					test_specific_url = ""
					total_pages = 99999
					print("[INFO] The specific page found!")
					break

		total_pages += 1

		print(f"\n[INFO] Checking page: {page_index} | Link: {page['link']}" )
		url = page['link']
		first = True
		auto_re = False
		auto_response_count = 0

		while (re or first):
			first = False
			
			try: 
				time.sleep(1)

				if (not re): 
	
					driver = get_driver()
					print("[INFO] Chrome driver initialized.")
					driver.get(url)
					print("[INFO] Page loaded.")

				re = False
				time.sleep(random.uniform(6, 12))
				body = driver.find_element(By.TAG_NAME, 'body')

				try: 
					content_extracted_by_json, data = extract_from_script()
					if not content_extracted_by_json: 
						content, author, description = extract_from_DOM()

						if author == "not found":
							try:
								author = driver.find_element(By.CLASS_NAME, "a_md_a").text.strip()
							except:
								author = "not found"
					else:
						content = data["content"]
						description = data["description"]
						raw_json_content = data["raw_json_content"] if data["raw_json_content"] != "" else "not found initially"

					try:
						author_meta = driver.find_element(By.CSS_SELECTOR, "meta[property='article:author']")
						author = author_meta.get_attribute("content")
					except Exception as e:
						print("[ERROR] A error occurred while extracting author name", flush=True)
						print("[INFO] Setting author to 'not found'", flush=True)
						print(e, flush=True)
						author = "not found"

				except Exception as e:
					print(f"[ERROR] A error occurred while extracting content", flush=True)
					raise

			except Exception as e:
				print(f"[ERROR] A error occurred while processing the page: {page_index}", flush=True)
				print(f"[ERROR] Site: {url}", flush=True)
				print(e, flush=True)

				# response = input("Retry? y/n: ")
				time.sleep(10)

				if auto_response_count == 3:
					response = "n"
				else:
					response = "y"
				auto_response_count += 1

				if response.lower() == 'y':
					print("[INFO] Retry...")
					re = True
				elif response.lower() == "n" or response == "":  
					# response = input("Description? Press enter to not add a description")
					response = "[AUTO] Limit of attempts reached"
					if not response:
							response = "No description provided"
					else: 
							print(f"[INFO] The user add a description: {response}")
					print("[INFO] Cancelling...")
					error_occurred = True
					page["Observation"] = "A error occurred while extracting content. The user cancelled the operation. Description: " + response
					re = False
				else:
					print("[INFO] The response is unknown.")
					print("[INFO] Cancelling...")
					error_occurred = True
					re = False

		if (not re):
			driver.quit()

		if error_occurred:
			content = "A error occurred while extracting content"
			author = "A error occurred while extracting content"
			description = "A error occurred while extracting content"

			pages_failed += 1
			error_occurred = False
		else:
			pages_succeeded += 1

		page["content"] = content
		page["author"] = author
		page["description"] = description
		page["raw_json_content"] = raw_json_content
		extracted_pages_with_content["pages"].append(page)

		content = ""
		author = ""
		description = ""
		raw_json_content = {}

		print("[INFO] Storing information in a JSON file...")
		with open(f"../archive/temp/pages_with_content/la_tercera/la_tercera_with_content-{page_index}.json", 'w', encoding='utf-8') as file:
			json.dump(extracted_pages_with_content, file, ensure_ascii=False, indent=4)

		extracted_pages_with_content = {"pages": []}

		driver.quit()

		time.sleep(random.uniform(6, 12))

# ----------------- Stats -----------------
print("\n\n")
print("----- STATS -----")

try:
	print(f"Amount of pages processed: {total_pages}")
	print(f"% Pages succeeded [{pages_succeeded}]: {round(100 * pages_succeeded/total_pages, 2)}")
	print(f"% Pages failed [{pages_failed}]: {round(100 * pages_failed/total_pages, 2)}")
except: 
	print("A error occurred while processing the stats")